In [40]:
# import libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import time
import json
import matplotlib.pyplot as plt
from shapely import wkt
#import pysal as ps
import shapefile
import censusdata
import censusgeocode as cg
import requests

In [41]:
#!pip install censusdata

In [42]:
#!pip install censusgeocode

In [43]:
#import all Seattle census tracts
tracts = gpd.read_file("Census Tract Data 2020/Census_2020_Tracts_with_PL_94-171_Redistricting__Data_for_1990-2020.shp")
tracts.head(1)

,OBJECTID_1,GEOID_20,TRACT_20_N,TRACT_20_L,TRACT_20,F1990_PL_d,F1990_PL_1,F1990_PL_2,F1990_PL_3,F1990_PL_4,...,F2020_PL22,F2020_PL23,F2020_PL24,F2020_PL25,F2020_PL26,AREA_ACRES,AREA_SQMI,Shape__Are,Shape__Len,geometry
0,1,53033000101,Census Tract 1.01,1.01,101,2145,1419,210,41,374,...,2040,1902,138,33,0,102.503713,0.160162,4.465062e+06,8662.10098,"POLYGON ((-122.29654 47.73198, -122.29653 47.7..."


In [44]:
print(tracts.crs)

epsg:4326


In [45]:
tract = tracts.to_crs('epsg:2927')

In [46]:
tract.columns.to_list()

['OBJECTID_1',
 'GEOID_20',
 'TRACT_20_N',
 'TRACT_20_L',
 'TRACT_20',
 'F1990_PL_d',
 'F1990_PL_1',
 'F1990_PL_2',
 'F1990_PL_3',
 'F1990_PL_4',
 'F1990_PL_5',
 'F1990_PL_6',
 'F1990_PL_7',
 'F1990_PL_8',
 'F1990_PL_9',
 'F1990_PL10',
 'F1990_PL11',
 'F1990_PL12',
 'F1990_PL13',
 'F1990_PL14',
 'F1990_PL15',
 'F1990_PL16',
 'F1990_PL17',
 'F1990_PL18',
 'F1990_PL19',
 'F1990_PL20',
 'F1990_PL21',
 'F2000_PL_d',
 'F2000_PL_1',
 'F2000_PL_2',
 'F2000_PL_3',
 'F2000_PL_4',
 'F2000_PL_5',
 'F2000_PL_6',
 'F2000_PL_7',
 'F2000_PL_8',
 'F2000_PL_9',
 'F2000_PL10',
 'F2000_PL11',
 'F2000_PL12',
 'F2000_PL13',
 'F2000_PL14',
 'F2000_PL15',
 'F2000_PL16',
 'F2000_PL17',
 'F2000_PL18',
 'F2000_PL19',
 'F2000_PL20',
 'F2000_PL21',
 'F2000_PL22',
 'F2000_PL23',
 'F2000_PL24',
 'F2000_PL25',
 'F2000_PL26',
 'F2010_PL_d',
 'F2010_PL_1',
 'F2010_PL_2',
 'F2010_PL_3',
 'F2010_PL_4',
 'F2010_PL_5',
 'F2010_PL_6',
 'F2010_PL_7',
 'F2010_PL_8',
 'F2010_PL_9',
 'F2010_PL10',
 'F2010_PL11',
 'F2010_PL12',

In [47]:
tract_df = tract[["GEOID_20","F2020_PL_d","AREA_ACRES","TRACT_20","geometry"]]

In [48]:
tract_df = tract_df.rename(index=str, columns = {'GEOID_20': 'GEOID','F2020_PL_d': 'population', 'TRACT_20': 'tract'})


In [49]:
tract_df.head()

,GEOID,population,AREA_ACRES,tract,geometry
0,53033000101,3759,102.503713,101,"POLYGON ((1198280.286 879815.173, 1198288.869 ..."
1,53033000102,4321,596.442465,102,"POLYGON ((1198268.837 877820.803, 1198710.196 ..."
2,53033000201,4416,446.424496,201,"POLYGON ((1191537.977 876074.777, 1191548.225 ..."
3,53033000202,4099,365.710684,202,"POLYGON ((1194973.723 880600.170, 1195306.262 ..."
4,53033000300,2820,299.225793,300,"POLYGON ((1186323.563 880876.512, 1186351.366 ..."


In [2]:
censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '53'), ('county', '033'), ('block group', '*')]), ['B01001_001E', 'B01002_001E', 'B19013_001E'])

NameError: name 'censusdata' is not defined

In [10]:
censusdata.geographies(censusdata.censusgeo([('state', '53'), ('county', '033')]), 'acs5', 2019)

{'King County, Washington': censusgeo((('state', '53'), ('county', '033')))}

In [11]:
# Download ACS 2013-2017 5-year estimates for King County, Washington.
kingcounty = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '53'), ('county', '033'), ('block group', '*')]),
                                   ['B01001_001E', 'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E',
                                    'B01001_024E', 'B01001_025E', 'B01001_044E', 'B01001_045E', 'B01001_046E',
                                    'B01001_047E', 'B01001_048E', 'B01001_049E'])

In [14]:
kingcounty.head(1)

,B01001_001E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,block group,NAME
"Block Group 2, Census Tract 257.02, King County, Washington: Summary level: 150, state:53> county:033> tract:025702> block group:2",1073,0,22,20,17,10,10,12,40,21,5,40,15,53,033,025702,2,"Block Group 2, Census Tract 257.02, King Count..."


In [13]:
# export census dataframe
censusdata.export.exportcsv('Data/sea_census.csv', kingcounty)

In [51]:
tract_df.to_csv('Data/all_demographics.csv')